# TSA Chapter 3: ACF Patterns: Stationary vs Non-Stationary

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_acf_nonstationary/TSA_ch3_acf_nonstationary.ipynb)

This notebook demonstrates:
- ACF comparison: slow decay for I(1) random walk vs fast decay for stationary AR(1)
- Differencing restores quick ACF decay.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 500
nlags = 30

eps = np.random.normal(0, 1, n)

# Stationary AR(1)
ar1 = np.zeros(n)
for t in range(1, n):
    ar1[t] = 0.7 * ar1[t-1] + eps[t]

# Random walk (I(1))
rw = np.cumsum(eps)

# First difference of RW (stationary)
diff_rw = np.diff(rw)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 7))

series = [ar1, rw, diff_rw]
titles = ['Stationary AR(1)', 'Random Walk I(1)', 'Differenced RW (stationary)']
colors = ['#2E7D32', '#DC3545', '#1A3A6E']

for i, (s, title, c) in enumerate(zip(series, titles, colors)):
    axes[0, i].plot(s, color=c, linewidth=0.8)
    axes[0, i].set_title(f'{title}', fontweight='bold')
    axes[0, i].set_xlabel('Time')

    acf_vals = acf(s, nlags=nlags)
    axes[1, i].bar(range(len(acf_vals)), acf_vals, color=c, alpha=0.7, width=0.6)
    ci = 1.96 / np.sqrt(len(s))
    axes[1, i].axhline(ci, color='red', linestyle='--', linewidth=0.8)
    axes[1, i].axhline(-ci, color='red', linestyle='--', linewidth=0.8)
    axes[1, i].set_title(f'ACF: {title}', fontweight='bold')
    axes[1, i].set_xlabel('Lag')
    axes[1, i].set_ylabel('ACF')

plt.tight_layout()
save_chart(fig, 'ch3_acf_nonstationary')
plt.show()

print("ACF Signatures:")
print("  Stationary: ACF decays quickly to zero")
print("  Non-stationary (I(1)): ACF decays VERY slowly, stays near 1")
print("  After differencing: ACF decays quickly (now stationary)")